In [262]:
df1 = pd.read_json(path_to_dev)
add_comments(df1, 'dv_2')

["My favorite character is Rocky because he's like a real person. He also has a lot of humorous lines.",
 'I think it was a pretty good entry in the Rocky series, not as good as the first film though.',
 'My favorite character was Paulie.',
 "One of the best sequels ever made. I'm not joking.",
 'Do you remember why Mister T was angry at Rocky?']

In [265]:
import json
import pandas as pd
import random

random.seed(3)
state = random.getstate()

def add_comments(df, chat_idx):
    # return df.iloc[idx].loc['all_documents'].get('comments')
    #df[df['chat_id'] == chat_idx][0]
    #return random.sample(list(df.iloc[idx].loc['all_documents'].get('comments')), 5)
    random.setstate(state)
    candidates = set(df[df['chat_id'] == chat_idx].iloc[0].loc['all_documents'].get('comments'))
    return random.sample(candidates, min(len(candidates),5))
    
def add_history(df, idx):
    retval = []
    if not df.full_history[idx] == ['NH']:
        retval = df.full_history[idx].copy()
    retval.append(df['query'][idx])
    return retval

def generate_candidates(df, idx):
    imdb_idx = df.iloc[idx]['imdb_id']
    temp = df[df['imdb_id'] != imdb_idx]
    #print(list(temp.index))
    retval = []
    random.setstate(state)
    for i in random.sample(list(temp.index), 2):
        retval.append(df.iloc[i].loc['response'])
    retval.append(df.iloc[idx].loc['response'])
    return retval

def get_utterance(df, idx):
    return {
        'candidates': generate_candidates(df, idx),
        'history': add_history(df, idx)
            }

def get_utterance_list(df, chat_idx):
    retval = []
    index = list(df[df['chat_id'] == chat_idx].index)
    #print(index)
    for idx in index:
        retval.append(get_utterance(df, idx))
        
    return retval

def get_data_point(df, chat_idx):
    return {
                'personality': add_comments(df, chat_idx),
                'utterances': get_utterance_list(df, chat_idx)
            }

    
def transform_holle_data_by_path(path_to_df):
    df = pd.read_json(path_to_df)
    
    #iterate thorugh chat_ids
    chat_ids = set(df['chat_id'])
    data = []
    for chat_id in chat_ids:
        data.append(get_data_point(df, chat_id))
    
    return data

def transform_holle_data(path_to_train, path_to_dev):
    
    return {'train': transform_holle_data_by_path(path_to_train),
            'valid': transform_holle_data_by_path(path_to_dev)}


    


In [266]:
%%time
path_to_train = '../holle/raw_data/train_data.json'
path_to_dev = '../holle/raw_data/dev_data.json'
data = transform_holle_data(path_to_train, path_to_dev)

CPU times: user 21min 48s, sys: 1.05 s, total: 21min 49s
Wall time: 21min 49s


In [268]:
len(data['train'])

7226

In [269]:
len(data['valid'])

930

In [267]:
%%time
path_to_output = '../holle/huggingface_data.json'
with open(path_to_output, 'w') as file:
    json.dump(data, file)

CPU times: user 532 ms, sys: 24 ms, total: 556 ms
Wall time: 554 ms


In [270]:
data['valid'][0].keys()

dict_keys(['personality', 'utterances'])

In [383]:
len(data['valid'][0]['personality'])
random.sample(data['valid'][2]['personality'], 5)

['I think it was overrated.',
 'I think it was not a good movie, and it did not do the character of Spider-Man justice.',
 "My favorite character was Peter Parker, because of all the memes that came out of Tobey Maguire's performance.",
 'I liked the one in which Spider Man kisses Mary Jane.',
 'My favorite character was Peter Parker.']

In [390]:
data['valid'][0]['personality']

{'Aside from a shaky final act, "Insidious" is a very scary and very fun haunted house thrill ride.',
 'Do you remember who was dreaming out of their body and wondering or floating out of their body.',
 'I just adore this movie, its the first one to scare me in a very long time and even repeat watches make me jump!',
 'I liked the one in which the baby toys were playing by themselves.',
 'I think it was awesome.',
 'I will have trouble sleeping tonight...',
 'My favorite character was the older lady who was able to see things no one else can.',
 "Scariest Movie I've Seen In Years!",
 'So scary...',
 'This is one of them horror films you enjoy but only the first time.',
 'When it came out it was the best movie like it in a long time. I thought it was pretty well done.',
 '[t]he film begins with promise.'}

In [ ]:
I just adore this movie, its the first one to scare me in a very long time and even repeat watches make me jump!
My favorite character was the older lady who was able to see things no one else can.
Aside from a shaky final act, "Insidious" is a very scary and very fun haunted house thrill ride.
I will have trouble sleeping tonight…
Scariest Movie I've Seen In Years!
